In [ ]:
import numpy as np
import pandas as pd

from pyspark.ml.functions import predict_batch_udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, struct, array
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType, Union, Dict

In [ ]:
%env PYSPARK_PYTHON=C:\Users\Milosz\AppData\Local\pypoetry\Cache\virtualenvs\recsys-streaming-ml-Mj1TWbkU-py3.10\Scripts\python.exe

In [ ]:
%env PYSPARK_DRIVER_PYTHON=C:\Users\Milosz\AppData\Local\pypoetry\Cache\virtualenvs\recsys-streaming-ml-Mj1TWbkU-py3.10\Scripts\python.exe

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'

In [ ]:
KAFKA_BROKER_URL = "kafka0:9093"
RECOMMENDATIONS_TOPIC = "recommendations"
USER_ACTIONS_TOPIC = "users.actions"

In [ ]:
def predict_batch_fn():
    # load model from checkpoint
    import torch    
    device = torch.device("cuda")
    model = Net().to(device)
    checkpoint = load_checkpoint(checkpoint_dir)
    model.load_state_dict(checkpoint['model'])

    # define predict function in terms of numpy arrays
    def predict(inputs: np.ndarray) -> np.ndarray:
        torch_inputs = torch.from_numpy(inputs).to(device)
        outputs = model(torch_inputs)
        return outputs.cpu().detach().numpy()
    
    return predict

In [ ]:
spark = SparkSession.builder \
        .appName("KafkaRead") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
        .getOrCreate()

In [ ]:
schema = StructType([
        StructField("user_id", StringType(), True)
    ])

In [ ]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BROKER_URL) \
    .option("subscribe", RECOMMENDATIONS_TOPIC) \
    .option("startingOffsets", "latest") \
    .load()

In [ ]:
df.printSchema()

In [ ]:
values_df = df.selectExpr("CAST(value AS STRING) as json_data") \
                .select(from_json(col("json_data"), schema).alias("data")) \
                .select("data.*")

In [ ]:
values_df.writeStream.format('console').outputMode('append').start().awaitTermination()

In [ ]:
query = values_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='30 seconds') \
    .start()

In [ ]:
query.awaitTermination(30)

In [ ]:
df_parsed = df.selectExpr("CAST(value AS STRING) as json_value") \
    .select(from_json(col("json_value"), schema).alias("data")) \
    .select("data.*")


In [ ]:
query = df_parsed.writeStream.outputMode("append").format("console").start()
query.awaitTermination()

In [ ]:
from recsys_streaming_ml.spark.utils import spark

def create_dataframe_from_dict(spark, data):
    """
    Create a DataFrame from a list of dictionaries.
    Each dictionary represents a record with a single field `user_id`.
    """
    schema = StructType([StructField("user_id", StringType(), True)])
    df = spark.createDataFrame(data, schema)
    return df

# Sample data
data = [
    {"user_id": "A1"},
    {"user_id": "B2"},
    {"user_id": "C3"},
    {"user_id": "D4"}
]

# Create Spark session
session = spark()

# Create DataFrame from data
df = create_dataframe_from_dict(session, data)

# Show DataFrame
#df.show()

In [ ]:
df.show()

In [ ]:
import pandas as pd
import pyspark as ps
import numpy as np
from pyspark.sql.types import StructType, StructField, IntegerType

def create_item_feature_store(spark):
    data = [(i, i) for i in range(100)]
    schema = StructType([
        StructField("parent_asin", IntegerType(), True),
        StructField("store_id", IntegerType(), True)
    ])
    df = spark.createDataFrame(data, schema)
    return df

item_feature_store = create_item_feature_store(session)

In [ ]:
item_feature_store.show()

In [ ]:
from recsys_streaming_ml.db import mongo_db, read_df_from_mongo
from recsys_streaming_ml.data.utils import load_feature_maps
import random

feature_maps = load_feature_maps("../.data/feature_maps.pkl")

def read_item_feature_store(db, feature_maps, collection='metadata'):
    item_feature_store_raw = read_df_from_mongo(db=db, collection=collection)
    item_feature_store = item_feature_store_raw.copy()
    item_feature_store['parent_asin'] = item_feature_store['parent_asin'].map(feature_maps['parent_id_map'])
    item_feature_store['store_id'] = item_feature_store['store'].map(feature_maps['store_id_map'])
    item_feature_store = item_feature_store.drop(columns='store').dropna().astype(int).sort_values(by='parent_asin').reset_index(drop=True)

    return item_feature_store

In [ ]:
item_feature_store = read_item_feature_store(mongo_db, feature_maps)

In [ ]:
item_feature_store = session.createDataFrame(item_feature_store)

In [ ]:
item_feature_store.show()

In [ ]:
from pyspark.sql.functions import col, create_map, lit, udf
from itertools import chain

user_id_mapping = {"A1": 2, "B2": 3, "C3": 1, "D4": 0}
rev_user_id_mapping = {v:k for k,v in user_id_mapping.items()}
rev_asin_mapping = {v[0]:f'ID_{v[0]}' for v in item_feature_store.select('parent_asin').distinct().collect()}

def process_data(
        df: ps.sql.dataframe.DataFrame, 
        item_feature_store: ps.sql.dataframe.DataFrame, 
        user_id_mapping: dict[str, int]
    ) -> ps.sql.dataframe.DataFrame:
    """
    Process the DataFrame by mapping user_ids using the provided dictionary.
    """
    mapping_expr = create_map([lit(x) for x in chain(*user_id_mapping.items())])

    processed_df = df.withColumn("map_user_id", mapping_expr[col("user_id")])
    processed_df = processed_df.crossJoin(item_feature_store)
    processed_df = processed_df.select("map_user_id", "parent_asin", "store_id")

    return processed_df

In [ ]:
processed_df = process_data(df, item_feature_store, user_id_mapping)

In [ ]:
processed_df.show()

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, collect_list

def get_ranked_topk_predictions(df, k=5):
    window = Window.partitionBy("map_user_id").orderBy(col("predicted_rating").desc())

    # Add a rank column to rank the rows within each partition by 'sum'
    ranked_predictions = df.withColumn("rank", rank().over(window))

    # Filter to keep only the top 5 'asin' values for each 'map_user_id'
    top_k = ranked_predictions.filter(col("rank") <= k)

    return top_k

def remap_entities(df, user_id_mapping, asin_mapping):
        mapping_expr_user = create_map([lit(x) for x in chain(*user_id_mapping.items())])
        mapping_expr_asin = create_map([lit(x) for x in chain(*asin_mapping.items())])

        df = df.withColumn("user_id", mapping_expr_user[col("map_user_id")])
        df = df.withColumn("asin", mapping_expr_asin[col("map_user_id")])

        return df.select("user_id", "asin", "rank")

def list_recommendations(df):
    # Aggregate the top k 'asin' values into a list for each 'map_user_id'
    result = df.groupBy("user_id").agg(collect_list("asin").alias("top_k_asins"))
    return result

In [ ]:
ranked_topk = get_ranked_topk_predictions(predictions)
remapped_ranked_topk = remap_entities(ranked_topk, rev_user_id_mapping, rev_asin_mapping)
recommendation_lists = list_recommendations(remapped_ranked_topk)

In [ ]:
ranked_topk.show()

In [ ]:
remapped_ranked_topk.show()

In [ ]:
recommendation_lists.show()

In [ ]:
df = spark.read.parquet("/path/to/test/data")
preds = df.withColumn("preds", mnist('data')).collect()

query = df_parsed.writeStream \
    .outputMode("append") \
    .format("console") \
    #.trigger(processingTime='15 seconds') \
    .start()

query.awaitTermination()

In [ ]:
pd.read_csv("../.data/dataset/train_data.csv")

In [ ]:
from recsys_streaming_ml.data.utils import load_feature_maps
import random

feature_maps = load_feature_maps("../.data/feature_maps.pkl")

random_user_ids = random.choices(list(feature_maps['user_id_map'].keys()), k=50)
pd.DataFrame(random_user_ids, columns=['user_id']).to_csv("../.data/sample_user_ids.csv", index=False)

In [63]:
import torch


#model = torch.jit.load("C:/Users/Milosz/Projects/recsys-streaming/recsys-streaming-ml/.runs/DeepFM/2024-05-11_15-29-20/model.pt", map_location='cpu')
model_input = torch.randint(0, 40, (1, 3)).to(torch.float)


In [79]:
model = DeepFM(emb_dim=8, hidden_dim=[32, 24, 10], feature_sizes=[100, 100, 100])
model.eval()
model_input = torch.randint(0, 40, (1, 3)).to(torch.float)
traced_script_module = torch.jit.trace(model, model_input)

# Save the TorchScript model
traced_script_module.save("../.model_repository/DeepFM/1/model.pt")

In [53]:
from recsys_streaming_ml.model.model import DeepFM
# model = torch.load("C:/Users/Milosz/Projects/recsys-streaming/recsys-streaming-ml/.runs/DeepFM/2024-05-11_15-29-20/model.pt")

In [66]:

model.eval()

DeepFM(
  (V): EmbeddingNet(
    (embeddings): ModuleDict(
      (0): Embedding(100, 8)
      (1): Embedding(100, 8)
      (2): Embedding(100, 8)
    )
  )
  (fm): FM()
  (dnn): MLP(
    (layers): Sequential(
      (0): Linear(in_features=24, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=32, out_features=24, bias=True)
      (5): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=24, out_features=10, bias=True)
      (9): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_features=10, out_features=1, bias=True)
    )
  )
)

In [78]:
torch.save(model, "../.model_repository/DeepFM/1/model.pt")

In [69]:
torch.onnx.export(model, model_input, "../.model_repository/DeepFM/1/model.onnx")

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, PandasUDFType, struct, col, array
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import ArrayType, FloatType, Union, Dict, StructType, StructField, DataType

import pandas as pd
import numpy as np
import tritonclient.http as httpclient
from functools import partial

from recsys_streaming_ml.spark.utils import spark

In [21]:
# Initialize Spark session
session = spark()

In [22]:
# Define the PySpark DataFrame
schema = StructType([
    StructField("input", ArrayType(FloatType()), False)
])

data = [(np.random.rand(3).astype(np.float32).tolist(),) for _ in range(100)]
df = session.createDataFrame(data, schema)
df.show()

+--------------------+
|               input|
+--------------------+
|[0.8619455, 0.431...|
|[0.18523002, 0.31...|
|[0.49610206, 0.59...|
|[0.04170201, 0.68...|
|[0.8219223, 0.164...|
|[0.17303495, 0.22...|
|[0.67281836, 0.80...|
|[0.42195797, 0.51...|
|[0.03965295, 0.75...|
|[0.27317488, 0.67...|
|[0.19653319, 0.85...|
|[0.90088624, 0.74...|
|[0.5246734, 0.251...|
|[0.31020227, 0.56...|
|[0.69740117, 0.87...|
|[0.6936094, 0.151...|
|[0.6217257, 0.591...|
|[0.81691366, 0.27...|
|[0.38775757, 0.60...|
|[0.41836286, 0.24...|
+--------------------+
only showing top 20 rows



In [24]:
# Triton client setup
TRITON_URL = 'localhost:8000'
MODEL_NAME = 'DeepFM'

def evaluate_model(batch_df: pd.DataFrame) -> pd.DataFrame:
    inputs = batch_df['input'].to_list()
    
    triton_client = httpclient.InferenceServerClient(url=TRITON_URL)
    
    input_tensor = httpclient.InferInput('input', inputs[0].shape, 'FP32')
    input_tensor.set_data_from_numpy(np.array(inputs))
    
    outputs = []
    for input_data in inputs:
        input_tensor.set_data_from_numpy(input_data)
        result = triton_client.infer(model_name=MODEL_NAME, inputs=[input_tensor])
        output_data = result.as_numpy('output')
        outputs.append(output_data)
    
    return pd.DataFrame({'output': outputs})

# Register UDF with Spark
evaluate_model_udf = pandas_udf(evaluate_model, returnType='output_col DataType', functionType=PandasUDFType.SCALAR)


c:\Users\Milosz\AppData\Local\pypoetry\Cache\virtualenvs\recsys-streaming-ml-Mj1TWbkU-py3.10\lib\site-packages\pyspark\sql\pandas\functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


ParseException: 
[UNSUPPORTED_DATATYPE] Unsupported data type "DATATYPE".(line 1, pos 11)

== SQL ==
output_col DataType
-----------^^^


In [27]:
TRITON_GRPC_URL = 'localhost:8001'
MODEL_NAME = 'DeepFM'


def triton_fn(triton_uri, model_name):
    import numpy as np
    import tritonclient.grpc as grpcclient
    
    np_types = {
      "BOOL": np.dtype(np.bool8),
      "INT8": np.dtype(np.int8),
      "INT16": np.dtype(np.int16),
      "INT32": np.dtype(np.int32),
      "INT64": np.dtype(np.int64),
      "FP16": np.dtype(np.float16),
      "FP32": np.dtype(np.float32),
      "FP64": np.dtype(np.float64),
      "FP64": np.dtype(np.double),
      "BYTES": np.dtype(object)
    }

    client = grpcclient.InferenceServerClient(triton_uri)
    model_meta = client.get_model_metadata(model_name)
    
    def predict(inputs):
        if isinstance(inputs, np.ndarray):
            # single ndarray input
            request = [grpcclient.InferInput(model_meta.inputs[0].name, inputs.shape, model_meta.inputs[0].datatype)]
            request[0].set_data_from_numpy(inputs.astype(np_types[model_meta.inputs[0].datatype]))
        else:
            # dict of multiple ndarray inputs
            request = [grpcclient.InferInput(i.name, inputs[i.name].shape, i.datatype) for i in model_meta.inputs]
            for i in request:
                i.set_data_from_numpy(inputs[i.name()].astype(np_types[i.datatype()]))
        
        response = client.infer(model_name, inputs=request)
        
        if len(model_meta.outputs) > 1:
            # return dictionary of numpy arrays
            return {o.name: response.as_numpy(o.name) for o in model_meta.outputs}
        else:
            # return single numpy array
            return response.as_numpy(model_meta.outputs[0].name)
        
    return predict

recommender = predict_batch_udf(partial(triton_fn, triton_uri=TRITON_GRPC_URL, model_name=MODEL_NAME),
                          input_tensor_shapes=[[3]],
                          return_type=ArrayType(FloatType()),
                          batch_size=128)

In [29]:
results_df = df.withColumn('output', recommender(df['input']))

# Show results
results_df.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "c:\Users\Milosz\AppData\Local\pypoetry\Cache\virtualenvs\recsys-streaming-ml-Mj1TWbkU-py3.10\lib\site-packages\pyspark\ml\functions.py", line 765, in predict
    predict_fn = make_predict_fn()
  File "C:\Users\Milosz\AppData\Local\Temp\ipykernel_26864\857280881.py", line 23, in triton_fn
  File "C:\Users\Milosz\AppData\Local\pypoetry\Cache\virtualenvs\recsys-streaming-ml-Mj1TWbkU-py3.10\lib\site-packages\tritonclient\grpc\_client.py", line 522, in get_model_metadata
    raise_error_grpc(rpc_error)
  File "C:\Users\Milosz\AppData\Local\pypoetry\Cache\virtualenvs\recsys-streaming-ml-Mj1TWbkU-py3.10\lib\site-packages\tritonclient\grpc\_utils.py", line 77, in raise_error_grpc
    raise get_error_grpc(rpc_error) from None
tritonclient.utils.InferenceServerException: [StatusCode.UNAVAILABLE] Request for unknown model: 'DeepFM' is not found


In [2]:
import pyspark
import sys
sys.path.append("../")


In [33]:
import os
os.environ['DISABLE_SPARK_ENVS'] = ""

In [32]:
from recsys_streaming_ml.spark.utils import spark

In [34]:
session = spark()

In [ ]:
recsys-streaming-ml\.data\dataset\dataset_spark.csv

In [ ]:
C:/Users/Milosz/Projects/recsys-streaming/recsys-streaming-ml/.data/dataset/dataset_spark.csv/part-00000-b5b5d215-6187-49ea-905e-70202bd20248-c000.csv

In [14]:
spark_df = session.read.option("header", "true").csv("C:/Users/Milosz/Projects/recsys-streaming/recsys-streaming-ml/.data/dataset/dataset_spark.csv/part-00000-b5b5d215-6187-49ea-905e-70202bd20248-c000.csv")

In [18]:
spark_df = session.read.option("header", "true").csv("C:/Users/Milosz/Projects/recsys-streaming/recsys-streaming-ml/.data/dataset/dataset_spark.csv/")

Py4JJavaError: An error occurred while calling o57.csv.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:180)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$m

In [26]:
"../" + DATASET_FILE.as_posix()

'../.data/dataset'

In [21]:
from recsys_streaming_ml.config import DATASET_FILE
import os
def find_csv_file(directory_path):
    # Iterate over the files in the directory
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            # Check if the file has a .csv suffix
            if file.endswith('.csv'):
                return os.path.join(root, file)
    return None

import pandas as pd
df = pd.read_csv(find_csv_file("../" + DATASET_FILE.as_posix()))

In [4]:
from recsys_streaming_ml.data.utils import load_feature_maps

In [6]:
maps = load_feature_maps('../.data/feature_maps.pkl')

In [11]:
maps.keys()

dict_keys(['user_id_map', 'parent_id_map', 'store_id_map'])

In [24]:
df.store.sort_values()

8029      0
8030      0
8031      0
8032      0
8028      0
       ... 
9355    376
9354    376
9353    376
9359    376
9335    376
Name: store, Length: 16201, dtype: int64

In [19]:
maps['store_id_map']

{'Crunchy Mama Box': 0,
 'Mechaly': 1,
 'CLUBTAC': 2,
 'The Dapper Dog Box': 3,
 'Coffee Beanery': 4,
 'Transcends Monthly Crystal Subscription': 5,
 'Mysteries in Time - adventures through history': 6,
 'Pusheen': 7,
 'Bitsbox': 8,
 'La Fraise Boutique Adalie': 9,
 'Beauteque': 10,
 'meowbox': 11,
 'MadrasIn': 12,
 "Well Read's Box of Books": 13,
 'MindfulSouls': 14,
 'VAHDAM': 15,
 'Geek Fuel EXP': 16,
 'Tinkering Toddler Crates': 17,
 'Autsy Box': 18,
 'Dayspa Body Basics': 19,
 'Bath Blessing Box': 20,
 'Avelas': 21,
 'RICK AND MORTY': 22,
 'SHIMAJIRO': 23,
 'Nature Gnaws': 24,
 'My Coffee and Book Club': 25,
 'KaffeeScape': 26,
 'goScribbler': 27,
 'Boujee Brown': 28,
 'Skivvie NIX': 29,
 'LiveGlam Inc.': 30,
 'Pressed': 31,
 'SOCK FANCY': 32,
 'Inked, LLC': 33,
 'Snoqualmie Falls Candy Shoppe': 34,
 'Enchanted Crystal': 35,
 'Loved Again Media': 36,
 'GetFaithbox': 37,
 'Chakra Box': 38,
 'Aroma Thyme': 39,
 'Trendy Apparel Shop': 40,
 'Loot Crate, Inc.': 41,
 'Driftaway Coffee':

In [20]:
df['store']

0        264
1        264
2        264
3        264
4        264
        ... 
16196     44
16197     44
16198     44
16199    102
16200    264
Name: store, Length: 16201, dtype: int64

In [16]:
spark_df.show()

+-----------+-------------+------+-------+-----+
|parent_asin|    timestamp|rating|user_id|store|
+-----------+-------------+------+-------+-----+
|          0|1639408671038|   2.0|   6551|  264|
|          0|1670094428392|   5.0|   3191|  264|
|          0|1653138455274|   5.0|  14957|  264|
|          0|1636644871638|   5.0|   8711|  264|
|          0|1639006187872|   5.0|   7138|  264|
|          0|1652340580030|   3.0|  12961|  264|
|          0|1633974025922|   5.0|   9479|  264|
|          0|1649532524800|   5.0|      0|  264|
|          0|1635107916926|   3.0|   3433|  264|
|          0|1634350509767|   5.0|   6782|  264|
|          0|1635187736470|   4.0|    298|  264|
|          0|1633827953703|   5.0|   2841|  264|
|          0|1666670589458|   5.0|  14892|  264|
|          0|1648242079936|   5.0|   1184|  264|
|          0|1640290884841|   5.0|  10483|  264|
|          0|1633800053883|   5.0|   3583|  264|
|          1|1565731866942|   1.0|  14233|   10|
|          1|1594956

In [27]:
from recsys_streaming_ml.db import read_df_from_mongo, mongo_db
df_ratings_historical = read_df_from_mongo(db=mongo_db, collection='ratings')

In [28]:
df_ratings_historical

,timestamp,rating,user_id,parent_asin
0,1602133857705,1.0,AEMJ2EG5ODOCYUTI54NBXZHDJGSQ,B09WC47S3V
1,1609110735433,2.0,AEEJBFZKUBEEMBZUZJV4UHFVEEBQ,B07QL1JRCN
2,1609937315319,1.0,AGSVZNZBTSGQBKZDZTQYEZHGDPCQ,B08N5QKX1Y
3,1616156351887,5.0,AFDERNB6BIR3U2DOR3S2KX7KJJXQ,B07KM6T8GV
4,1559533206066,5.0,AE6P2YJ6FKX332MD56GPJFSHXNJQ,B07NVKNVNM
...,...,...,...,...
16211,1608846627043,5.0,AGX4UTCIP4SGTHHDA4E3WGPIJFBQ,B09TMNNYWH
16212,1552764727421,1.0,AH4WBYSLGIAMVLCT2PPRTMNNJD6A,B07GNX34XV
16213,1575580196049,5.0,AHUTNTBX5E65YNIHI2ERA2AMFM4Q,B07VXY755K
16214,1656510163413,1.0,AF6RXEUYBSSOMO5RSS23OPB5QKSA,B07VXY755K


In [35]:
from pyspark.sql.types import StructType, StructField, StringType

data = [
    ("B01M7UD21X", "AFRAYYOGKVOG5B2SGEOLZTPLCSXQ"),
    ("B0BM2J47PC", "AE73UZUFL544D3FROI53QWXB5B5A"),
    ("B07MGJZ2MH", "AGU7FC2GHGCOSOAYWM2USUTBPLBA"),
    ("B07NVL6TJG", "AHW5ZMORJKQXUF4BASUKDM3XB6QA"),
    ("B07QD9WBGB", "AG5BN7XS32ZFCNJT3BR6Y43TM3HA"),
    ("B07RBYJN37", "AGDVSQR73TP57VHWUACAIKV5JTGA"),
    ("B08DDK1TP5", "AGZQGMHGM7ISG2AP5J7W5TKBYANQ")
]

# Define the schema
schema = StructType([
    StructField("parent_asin", StringType(), True),
    StructField("user_id", StringType(), True)
])

# Create the DataFrame
df = session.createDataFrame(data, schema)

# Show the DataFrame
df.show()

+-----------+--------------------+
|parent_asin|             user_id|
+-----------+--------------------+
| B01M7UD21X|AFRAYYOGKVOG5B2SG...|
| B0BM2J47PC|AE73UZUFL544D3FRO...|
| B07MGJZ2MH|AGU7FC2GHGCOSOAYW...|
| B07NVL6TJG|AHW5ZMORJKQXUF4BA...|
| B07QD9WBGB|AG5BN7XS32ZFCNJT3...|
| B07RBYJN37|AGDVSQR73TP57VHWU...|
| B08DDK1TP5|AGZQGMHGM7ISG2AP5...|
+-----------+--------------------+



In [36]:
df.toPandas()

,parent_asin,user_id
0,B01M7UD21X,AFRAYYOGKVOG5B2SGEOLZTPLCSXQ
1,B0BM2J47PC,AE73UZUFL544D3FROI53QWXB5B5A
2,B07MGJZ2MH,AGU7FC2GHGCOSOAYWM2USUTBPLBA
3,B07NVL6TJG,AHW5ZMORJKQXUF4BASUKDM3XB6QA
4,B07QD9WBGB,AG5BN7XS32ZFCNJT3BR6Y43TM3HA
5,B07RBYJN37,AGDVSQR73TP57VHWUACAIKV5JTGA
6,B08DDK1TP5,AGZQGMHGM7ISG2AP5J7W5TKBYANQ


In [37]:
pd.concat([df.toPandas(), df_ratings_historical])

,parent_asin,user_id,timestamp,rating
0,B01M7UD21X,AFRAYYOGKVOG5B2SGEOLZTPLCSXQ,NaN,NaN
1,B0BM2J47PC,AE73UZUFL544D3FROI53QWXB5B5A,NaN,NaN
2,B07MGJZ2MH,AGU7FC2GHGCOSOAYWM2USUTBPLBA,NaN,NaN
3,B07NVL6TJG,AHW5ZMORJKQXUF4BASUKDM3XB6QA,NaN,NaN
4,B07QD9WBGB,AG5BN7XS32ZFCNJT3BR6Y43TM3HA,NaN,NaN
...,...,...,...,...
16211,B09TMNNYWH,AGX4UTCIP4SGTHHDA4E3WGPIJFBQ,1.608847e+12,5.0
16212,B07GNX34XV,AH4WBYSLGIAMVLCT2PPRTMNNJD6A,1.552765e+12,1.0
16213,B07VXY755K,AHUTNTBX5E65YNIHI2ERA2AMFM4Q,1.575580e+12,5.0
16214,B07VXY755K,AF6RXEUYBSSOMO5RSS23OPB5QKSA,1.656510e+12,1.0
